In [ ]:
from mplsoccer.pitch import Pitch
import numpy as np
import matplotlib.pyplot as plt
import cv2
from prompt_toolkit.shortcuts.progress_bar import Label

from data_parse import DataParser
from keypoint import Keypoint
import pandas as pd

In [ ]:
directory = 'data/SoccerNetGS/gamestate-2024/train/'
data_parser = DataParser(directory)
data = data_parser.parse_data_to_df()

In [ ]:
one_img_data = data[data['image_id'] == '1060000001']

In [ ]:
positional_data: pd.DataFrame = data_parser.get_shirts_data()

In [ ]:
positional_data

In [ ]:
pitch_data: pd.DataFrame = data_parser.get_pitch_data_df()

In [ ]:
pitch_data_one_img = pitch_data[pitch_data['image_id'] == 1060000233]

In [ ]:
keypoint = Keypoint()
lines = keypoint.get_func_pattern(pitch_data_one_img)

In [ ]:
lines

In [ ]:
keypoints = keypoint.get_all_keypoints_by_id(np.int64(1060000233))

In [ ]:
keypoints

In [ ]:
keypoints_not_none = [point for point in keypoints if point is not None]

In [ ]:
keypoints_np = np.array([[x, y] for x, y in keypoints_not_none])

In [ ]:
keypoints_np = keypoints_np[:4]

In [ ]:
keypoints_np

In [ ]:
image = cv2.imread('data/SoccerNetGS/gamestate-2024/train/SNGS-060/img1/000233.jpg')
for point in keypoints[-4:]:
    if point is not None:
        cv2.circle(image, (int(point[0]), int(point[1])), 5, (0, 0, 255), -1)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
real_pitch_coordinates = np.array([[16.5, 13.85], [0.0, 43.15], [0.0, 68.0], [16.5, 54.15]])

In [ ]:
print(real_pitch_coordinates)

In [ ]:
homography_mat, mask = cv2.findHomography(keypoints_np, real_pitch_coordinates)

In [ ]:
homography_mat

In [ ]:
# pith size in meters
pitch_length = 105
pitch_width = 68

In [ ]:
positional_data_one_img = positional_data[positional_data['image_id'] == '1060000233']
positional_data_one_img

In [ ]:
player_position_df = positional_data_one_img.copy()
player_position_df['x'] = player_position_df['bbox_image_x'] + player_position_df['bbox_image_w'] / 2

In [ ]:
player_position_df['y'] = player_position_df['bbox_image_y'] + player_position_df['bbox_image_h'] / 2

In [ ]:
player_position_df.drop(columns=['bbox_image_x', 'bbox_image_y', 'bbox_image_w', 'bbox_image_h'], inplace=True)

In [ ]:
player_position_df

In [ ]:
player_position_df_np = player_position_df.to_numpy()

In [ ]:
player_position_df_np

In [ ]:
points_src = player_position_df.drop(columns=['image_id', 'attributes_team']).to_numpy()

In [ ]:
points_src = np.array([[x, y] for x, y in points_src])

In [ ]:
points_src

In [ ]:
points_src = points_src.reshape(-1, 1, 2)

In [ ]:
points_dst = cv2.perspectiveTransform(points_src, homography_mat)

In [ ]:
points_dst = points_dst.reshape(-1, 2)

In [ ]:
points_dst

In [ ]:
points_dst[:, 1] = 68 - points_dst[:, 1]

In [ ]:
points_dst

In [ ]:
from mplsoccer import Pitch, Sbopen
import matplotlib.pyplot as plt

# read data
parser = Sbopen()
df, related, freeze, tactics = parser.event(7478)

In [ ]:
df = df[(df.player_name == 'Jodie Taylor') & (df.type_name == 'Pass')].copy()

In [ ]:
pitch = Pitch()
fig, ax = pitch.draw(figsize=(8, 6))
scatter = pitch.scatter(df.x, df.y, ax=ax, edgecolor='black', facecolor='cornflowerblue')
plt.show()

In [ ]:
team = player_position_df['attributes_team'].to_numpy()

In [ ]:
team = np.where(team == 'left', 1, 0)

In [ ]:
team

In [ ]:
pitch = Pitch(pitch_type='uefa', axis=True, label=True)
fig, ax = pitch.draw()
scatter = pitch.scatter(points_dst[:, 0], points_dst[:, 1], ax=ax, edgecolor="black", c=team)
plt.grid()

In [ ]:
plt.show()

In [ ]:
points_dst[1:, 1]

In [ ]:
pitch_length = 105
pitch_width = 68
corner_to_box_edge = 13.85
box_edge_to_goal = 16.5
goal_width = 7.32
inside_box_edge_to_goal = 5.5
box_length = 16.5
inside_box_length = 5.5
circle_radius = 9.15
big_box_edge_to_inside_box_edge = box_edge_to_goal - inside_box_edge_to_goal

real_pitch_coordinates_full = np.array([
    [0.0, 0.0],
    [0.0, corner_to_box_edge],
    [0.0, corner_to_box_edge + big_box_edge_to_inside_box_edge],
    [0.0, 68.0 - corner_to_box_edge - big_box_edge_to_inside_box_edge],
    [0.0, 68.0 - corner_to_box_edge],
    [0.0, 68.0],
    
    [inside_box_length, corner_to_box_edge + big_box_edge_to_inside_box_edge],
    [inside_box_length, 68 - corner_to_box_edge - big_box_edge_to_inside_box_edge],
    
    [11.0, pitch_width/2],
    
    [box_length, corner_to_box_edge],
    [box_length, 34.0 - 7.66],
    [box_length, 34.0 + 7.66],
    [box_length, 68 - corner_to_box_edge],
    
    [pitch_length/2 - circle_radius, pitch_width/2],
    
    [pitch_length/2, 0.0],
    [pitch_length/2, pitch_width/2 - circle_radius],
    [pitch_length/2, pitch_width/2 + circle_radius],
    [pitch_length/2, 68.0],
    
    [pitch_length/2 + circle_radius, pitch_width/2],
    
    [pitch_length - box_length, corner_to_box_edge],
    [pitch_length - box_length, 34.0 - 7.66],
    [pitch_length - box_length, 34.0 + 7.66],
    [pitch_length - box_length, 68 - corner_to_box_edge],
    
    [pitch_length - 11.0, pitch_width/2],
    
    [pitch_length - inside_box_length, corner_to_box_edge + big_box_edge_to_inside_box_edge],
    [pitch_length - inside_box_length, 68 - corner_to_box_edge - big_box_edge_to_inside_box_edge],
    
    [pitch_length, 0.0],
    [pitch_length, corner_to_box_edge],
    [pitch_length, corner_to_box_edge + big_box_edge_to_inside_box_edge],
    [pitch_length, 68.0 - corner_to_box_edge - big_box_edge_to_inside_box_edge],
    [pitch_length, 68.0 - corner_to_box_edge],
    [pitch_length, 68.0]
])

In [ ]:
real_pitch_coordinates_full